# Programming Language Classifier

Download Watson Developer Cloud, import libraries, and load train/test sets

In [4]:
!pip install --upgrade watson_developer_cloud
!pip install wget

Requirement already up-to-date: watson_developer_cloud in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: Twisted>=13.2.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson_developer_cloud)
Requirement not upgraded as not directly required: python-dateutil>=2.5.3 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson_developer_cloud)
Requirement not upgraded as not directly required: pyOpenSSL>=16.2.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson_developer_cloud)
Requirement not upgraded as not directly required: requests<3.0,>=2.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson_developer_cloud)
Requirement not upgraded as not directly required: service-identity>=17.0.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson_developer_cloud)
Requirement not upgraded as not directly required: autobahn>=0.10.9 in /opt/con

In [5]:
import os
import re
import csv
import json
import wget
import base64
import operator
import numpy as np
from os import listdir
from collections import *
from os.path import isfile, join
from watson_developer_cloud import NaturalLanguageClassifierV1

In [13]:
wget.download( 'https://github.com/IBM/programming-language-classifier/blob/master/data/githubtrainingdatacompressed.npz?raw=true' )
wget.download( 'https://github.com/IBM/programming-language-classifier/blob/master/data/githubtestdatacompressed.npz?raw=true' )
wget.download( 'https://github.com/IBM/programming-language-classifier/blob/master/data/trainingdata.csv?raw=true' )

train_data = np.array(np.load("githubtrainingdatacompressed.npz")['arr_0'])
test_data = np.array(np.load("githubtestdatacompressed.npz")['arr_0'])

### A little more preprocessing

break the training data into separate dictionaries indexed by pl type, and map training data to a csv for Watson (my csv is on GitHub)

In [14]:
pls = {}
for row in range(len(train_data)):
    if train_data[row][1] not in pls:
        pls[train_data[row][1]] = []
    pls[train_data[row][1]].append(train_data[row][0])
    

CSV cannot exceed 1024 characters for column width and 15000 rows

In [93]:
chunk = 1024
with open('trainingdata.csv', 'w') as csvfile:
    fieldnames = ['text', 'pl']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    for i in train_data:
        for j in range(0,len(i[0]),chunk):
            text = re.sub(' +',' '," ".join(re.split(r'[^\w]', re.sub(re.compile("/\*.*?\*/",re.DOTALL ) ,"" ,i[0][j:j+chunk]))))   
            writer.writerow({'text': text, 'pl': i[1]})

## Naive Bayes Classifier

Here we train a Naive Bayes Classifier
for a light review on Naive Bayes look through the slides on GitHub
for a thorough background on this topic (and many others in Machine Learning) 
check out Tom Mitchell's Carnegie Mellon course 
http://cc-web.isri.cmu.edu/CourseCast/Viewer/Default.aspx?id=a666b6e6-ad23-4fa3-96ce-ae50a42f45a3

In [10]:
def bayes_train(pldict, samples):
    plprobs = {}
    counts = Counter()
    for i in pldict.iterkeys():
        plprobs[i] = float(len(pldict[i]))/samples
        
    plwordprobs = {}
    plwordcounts = {}
    for pl in pldict.iterkeys():
        plwordprobs[pl] = {}
        plwordcounts[pl] = 0
    
    for pl in pldict.iterkeys():
        for i in pldict[pl]:
            counts.update(filter(None, re.split(r'[^\w]', re.sub(re.compile("/\*.*?\*/",re.DOTALL ) ,"" ,i))))
            for word in counts.iterkeys():
                if word not in plwordprobs[pl]:
                    plwordprobs[pl][word] = counts[word]
                else:
                    plwordprobs[pl][word] += counts[word]
                plwordcounts[pl] += counts[word]
            plwordcount = 0
            counts = Counter()
    for pl in plwordprobs.iterkeys():   
        for word in plwordprobs[pl]:
            plwordprobs[pl][word] = float(plwordprobs[pl][word])/plwordcounts[pl]
        
    
    return plprobs, plwordprobs
    
plrobs, plwordprobs = bayes_train(pls, len(training_data))

Checking out the distribution of programming languages in our training set, and 10 of the most commonly used words of a particular language, try replacing 'sh' with other languages and observe the output

In [161]:
plprobs

{'go': 0.04184782608695652,
 'java': 0.2184782608695652,
 'js': 0.20597826086956522,
 'm': 0.06956521739130435,
 'py': 0.07880434782608696,
 'sh': 0.17771739130434783,
 'swift': 0.175,
 'xml': 0.03260869565217391}

In [153]:
sorted(plwordprobs['sh'].iteritems(), key=operator.itemgetter(1) ,reverse=True)[:10]

[('echo', 0.03864587716931944),
 ('the', 0.03778674579624146),
 ('0', 0.033368860713964506),
 ('1', 0.030890171747144934),
 ('if', 0.02780092617570351),
 ('for', 0.027205853790126674),
 ('import', 0.02685395198273526),
 ('to', 0.02617263735275594),
 ('in', 0.02470367772682303),
 ('return', 0.023925887959715765)]

Using the Naive Bayes Classifier to predict on the test set, again use the CMU course as a reference

In [17]:
def testbayes(testdata,plprob,plwordprob):
    Ypred = []

    for row in testdata:
        testcounter = Counter()
        testcounter.update(filter(None, re.split(r'[^\w]', re.sub(re.compile("/\*.*?\*/",re.DOTALL ) ,"" ,str(row[0])))))

        prob = {}
        for key in plprob.iterkeys():
            prob[key] = 0
        for key in prob.iterkeys():
            for i in testcounter:
                if i not in plwordprobs[key]:
                    plwordprob[key][i] = 1e-4
                else:
                    plwordprob[key][i] += 1e-4
                prob[key] += testcounter[i]*np.log(plwordprob[key][i])
            prob[key] += np.log(plprob[key])
        Ypred.append(max(prob.iteritems(), key=operator.itemgetter(1))[0])
    
    return Ypred

In [20]:
predictions = testbayes(test_data, plprobs, plwordprobs)

### Watson and Evaluating Classification Accuracy

Autherticate with Watson, send it the training data csv, wait for it to finish its training phase, and compute the accuarcy of both models. 

My results are displayed below. Let me know what you get at nacosta@us.ibm.com

In [104]:
natural_language_classifier = NaturalLanguageClassifierV1(
    username="YOURUSERNAME",
    password="YOURPASSWORD")

In [117]:
with open('trainingdatanew.csv', 'rb') as training_data:
    print(json.dumps(natural_language_classifier.create(training_data=training_data, name='codepattern'), indent=2))

{
  "status": "Training", 
  "name": "pydata", 
  "language": "en", 
  "created": "2017-11-26T05:28:14.097Z", 
  "url": "https://gateway.watsonplatform.net/natural-language-classifier/api/v1/classifiers/ee2ec4x254-nlc-4275", 
  "status_description": "The classifier instance is in its training phase, not yet ready to accept classify requests", 
  "classifier_id": "ee2ec4x254-nlc-4275"
}


Copy/Paste your classifier_id into the variable below. In the above example, "ee2ec4x254-nlc-4275" would be used. Monitor the status of your classifer be using the API below. Once the classifier's "status_description" matches the example below, proceed.

In [ ]:
classifier_id = "YOURCLASSIFIERID"

In [118]:
natural_language_classifier.status(classifier_id)

{u'classifier_id': u'ee2ec4x254-nlc-4275',
 u'created': u'2017-11-26T05:28:14.097Z',
 u'language': u'en',
 u'name': u'pydata',
 u'status': u'Training',
 u'status_description': u'The classifier instance is in its training phase, not yet ready to accept classify requests',
 u'url': u'https://gateway.watsonplatform.net/natural-language-classifier/api/v1/classifiers/ee2ec4x254-nlc-4275'}

In [154]:
natural_language_classifier.status("classifier_id")

{u'classifier_id': u'ee2ec4x254-nlc-4275',
 u'created': u'2017-11-26T05:28:14.097Z',
 u'language': u'en',
 u'name': u'pydata',
 u'status': u'Available',
 u'status_description': u'The classifier instance is now available and is ready to take classifier requests.',
 u'url': u'https://gateway.watsonplatform.net/natural-language-classifier/api/v1/classifiers/ee2ec4x254-nlc-4275'}

In [35]:
def compute_my_accuracy(pred, testdata):
    count = 0
    for i in range(len(pred)):
        if pred[i]==testdata[i][1]:
            count += 1
    return float(count)/len(pred)

In [129]:
def compute_watson_accuracy(testdata, classifier_id):
    count = 0
    for i in testdata:
        x = natural_language_classifier.classify(classifier_id,re.sub(' +',' '," ".join(re.split(r'[^\w]', re.sub(re.compile("/\*.*?\*/",re.DOTALL ) ,"" ,i[0]))))[0:1024])
        if x['top_class'] == i[1]:
            count += 1
    return float(count)/len(testdata)
    
def compute_watson_accuracy_fast(pred, testdata):
    count = 0
    for i in range(len(pred)):
        if pred[i]==testdata[i][1]:
            count += 1
    return float(count)/len(pred)
    
    

In [ ]:
watsonpred = []
for i in test_data:
    x = natural_language_classifier.classify(classifier_id,re.sub(' +',' '," ".join(re.split(r'[^\w]', re.sub(re.compile("/\*.*?\*/",re.DOTALL ) ,"" ,i[0]))))[0:1024])
    watsonpred.append(x['top_class']) 
    
print("My classifier's accuracy: " + str(compute_my_accuracy(predictions, test_data)))
print("Watson's accuracy: " + str(compute_watson_accuracy_fast(watsonpred, test_data)))